In [2]:
%matplotlib inline
import math
import numpy as np
import matplotlib.pyplot as plt
import pv_model_210131 as pv


In [3]:
Iscn = 8.7 # nominal short circuit current
Vocn = 37.7 # nominal open circuit voltage
Imp = 8.2 # array current at MPP
Vmp = 30.1 # array voltage at MPP
Pmax_e = Vmp * Imp # experimental array max power output
Kv = -0.0032 * Vocn # voltage temperature coefficient
Ki = -0.0032 * Iscn # current temperature coefficient
Ns = 60 # number of series cells
Gn = 1000 # nominal irradiance
Tn = 25 + 273.15 # nominal operating temperature

q = 1.602 * math.pow(10, -19) # charge on electron
k = 1.3806503 * math.pow(10, -23) # boltzmann constant


In [9]:
# I_i is the value of I 
def current(Ipv, I0, I_i, V, Vt, a, Rs, Rp):
    # using Iscn in place of I to calculate I
    math.exp((V+Rs*I_i)/(Vt*a)-1)
    I = Ipv - I0*(math.exp((V+Rs*I_i)/(Vt*a)-1)) - (V+Rs*I_i)/Rp
    return I

In [25]:

G = 500
T = 30 + 273.15
Vtn = k * Tn / q
Vt = k * T / q
deltaT = T - Tn
a = 1

Rs_max = (Vocn - Vmp) / Imp
Rp_min = Vmp / (Iscn - Imp) - Rs_max

Rp = Rp_min
Rs_vals = np.arange(0, Rs_max, 0.001)
itr = 0
v = np.arange(0, Vocn, 0.1)
i = np.arange(Iscn, 0, -0.1)

err = 10
Rp_prev = Rp

for Rs in Rs_vals:
    if Rp <= 0:
        print('less than zero')
        break
    V = v[itr]
    Ipvn = Iscn * (Rs+Rp)/Rp # why is Ipvn the current through Rp?
    Ipv = (Ipvn + Ki*deltaT) * G/Gn
    Isc = (Iscn + Ki*deltaT) * G/Gn
    I0n = (Ipv - Vocn/Rp) / (math.exp(Vocn/Vt/a/Ns)-1)
    I0 = I0n
    
    Rp_prev = Rp
    Rp = Vmp * (Vmp + Imp*Rs) / (Vmp * Ipv - Vmp* I0 * math.exp((Vmp+Imp*Rs)/Vt/Ns/a)+Vmp*I0-Pmax_e)
    
    I_p = Ipv - I0*(math.exp( q/(k*T)*(Vmp+Rs*Imp)/(a*Ns) - 1 ) - (Vmp+Rs*Imp)/Rp)
    Pmax_m = Vmp*I_p
    
    if abs(Pmax_m - Pmax_e) < err:
        break
    itr += 1

print(Rp_prev)
print(Pmax_m, Pmax_e)

less than zero
59.27317073170732
128.5235702755855 246.82


### Problems:
    - only using Vmp and Imp previously calculated, should be calculating it every iteration right?
    - Rp gets really negative suddenly
    - current equation gets out of range pretty fast
        - implement Newton-Raphson to calculate I?